In [1]:
import os 
import cv2
import glob
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.layers import Dense
from keras.models import Sequential
from tensorflow.keras import models
from tensorflow.keras import layers
from keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from keras.layers import Convolution2D,Dense,MaxPool2D,Activation,Dropout,Flatten
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
     

In [2]:
def get_files(directory):
  if not os.path.exists(directory):
    return 0
  count=0
  for current_path,dirs,files in os.walk(directory):
    for dr in dirs:
      count+= len(glob.glob(os.path.join(current_path,dr+"/*")))
  return count
train_dir = "D:\Mini Project Dataset\FInal Review\Potato dataset Train"
test_dir = "D:\Mini Project Dataset\FInal Review\Potato dataset Test"

In [3]:
train_samples =get_files(train_dir)
num_classes=len(glob.glob(train_dir+"/*"))
test_samples=get_files(test_dir)
print(num_classes,"Classes")
print(train_samples,"Train images")
print(test_samples,"Test images")

3 Classes
4721 Train images
431 Test images


In [5]:
import math
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
img_height = 224
img_width = 224
# Define data generator
datagen = ImageDataGenerator(    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)  # Scale pixel values to range [0, 1]

# Define batch size
batch_size = 32

# Flow images from directory and convert to numpy arrays
generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),  # Specify target size for resizing images
    batch_size=batch_size,
    class_mode=None,  # Do not generate class labels
    shuffle=False  # Keep images in order
)

# Number of images
num_images = generator.samples
num_batches = math.ceil(num_images / batch_size)
# Initialize numpy array to store images
image_arrays = np.zeros((num_images, img_height, img_width, 3))

# Iterate over generator to collect image arrays
for i in range(num_batches):
    batch = generator[i]
    start_index = i * batch_size
    end_index = min((i + 1) * batch_size, num_images)
    image_arrays[start_index:end_index] = batch

# Now, image_arrays contains numpy arrays of all images in train_dir


Found 4721 images belonging to 3 classes.


In [6]:
len(image_arrays)

4721

In [7]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data generator
datagen = ImageDataGenerator(rescale=1./255)  # Scale pixel values to range [0, 1]

# Define batch size
batch_size = 32

# Flow images from directory and convert to numpy arrays
generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),  # Specify target size for resizing images
    batch_size=batch_size,
    class_mode=None,  # Do not generate class labels
    shuffle=False  # Keep images in order
)

# Number of images
num_images = generator.samples

# Initialize numpy array to store images
image_arrays_test = np.zeros((num_images, img_height, img_width, 3))

# Initialize index counter
idx = 0

# Iterate over generator to collect image arrays
for batch in generator:
    start_index = idx
    end_index = idx + batch.shape[0]
    image_arrays_test[start_index:end_index] = batch
    idx = end_index
    
    if idx >= num_images:
        break  # Ensure loop ends when all images are collected

# Now, image_arrays contains numpy arrays of all images in train_dir


Found 431 images belonging to 3 classes.


In [8]:
len(image_arrays_test)

431

In [9]:
from sklearn.decomposition import PCA

# Assume you have your image data stored in X_train and X_test
# X_train and X_test should be numpy arrays where each row represents an image

# Step 1: Flatten the image data (assuming each image is a 2D array)
# Reshape each image to a vector
X_train_flattened = image_arrays.reshape(image_arrays.shape[0], -1)
X_test_flattened = image_arrays_test.reshape(image_arrays_test.shape[0], -1)

# Step 2: Normalize the image data (optional but recommended)
# It's common to normalize the pixel values to a range [0, 1] or [-1, 1]
X_train_flattened = X_train_flattened.astype('float32') / 255.0
X_test_flattened = X_test_flattened.astype('float32') / 255.0

# Step 3: Apply PCA for feature extraction and dimensionality reduction
# Specify the number of components (e.g., 100) to reduce to
n_components = 100
pca = PCA(n_components=n_components)

# Fit PCA on training data
pca.fit(X_train_flattened)

# Apply the transformation to both training and test data
X_train_pca = pca.transform(X_train_flattened)
X_test_pca = pca.transform(X_test_flattened)

In [8]:
from tensorflow.keras import models, layers

def create_cnn_model_extended():
    model = models.Sequential()
    
    # Upsampling Layer
    model.add(layers.UpSampling2D((4, 4), input_shape=(28, 28, 1)))
    
    # Convolutional Layers 1-5
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.25))
    
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.25))
    
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.25))
    
    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.25))
    
    model.add(layers.Conv2D(512, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.25))
    
    # Convolutional Layer 6
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.UpSampling2D((2, 2)))
    model.add(layers.Dropout(0.25))
    
    # Convolutional Layer 7
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.UpSampling2D((2, 2)))
    model.add(layers.Dropout(0.25))
    
    # Convolutional Layer 8
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.UpSampling2D((2, 2)))
    model.add(layers.Dropout(0.25))
    
    # Convolutional Layer 9
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(layers.UpSampling2D((2, 2)))
    model.add(layers.Dropout(0.25))
    
    # Output Layer
    model.add(layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same'))

    return model

# Create the extended CNN model
cnn_model_extended = create_cnn_model_extended()

# Display model summary
cnn_model_extended.summary()


/opt/conda/lib/python3.10/site-packages/keras/src/layers/reshaping/up_sampling2d.py:72: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ up_sampling2d (UpSampling2D)    │ (None, 112, 112, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 110, 110, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 55, 55, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 55, 55, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 53, 53, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 26, 26, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 26, 26, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 24, 24, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 10, 10, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 5, 5, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 5, 5, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 3, 3, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 1, 1, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 1, 1, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 1, 1, 256)      │     1,179,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_1 (UpSampling2D)  │ (None, 2, 2, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 2, 2, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 2, 2, 128)      │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_2 (UpSampling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 4, 4, 64)       │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_3 (UpSampling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 8, 8, 64)       │             

 Total params: 3,136,067 (11.96 MB)

 Trainable params: 3,136,067 (11.96 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import cv2

resized_images = [cv2.resize(img, (224, 224)) for img in image_arrays]

# Convert the list of resized images to a numpy array
resized_image_arrays = np.array(resized_images)

In [ ]:
cnn_model_extended.compile(optimizer='adam',loss = 'categorical_crossentropy',metrics=['accuracy'])
history1 = cnn_model_extended.fit(
    resized_image_arrays,#egitim verileri
    steps_per_epoch=None,
    epochs=35,
    validation_data=image_arrays_test,
    validation_steps=None,
    verbose=1,
    callbacks=[ReduceLROnPlateau(monitor='val_loss', factor=0.3,patience=3, min_lr=0.000001)],
    shuffle=True
    )

In [11]:
input_shape = (224,224,3)
model = Sequential()
model.add(Conv2D(32, (5, 5),input_shape=input_shape,activation='relu',name="conv2d_1"))
model.add(MaxPooling2D(pool_size=(3, 3),name="max_pooling2d_1"))
model.add(Conv2D(32, (3, 3),activation='relu',name="conv2d_2"))
model.add(MaxPooling2D(pool_size=(2, 2),name="max_pooling2d_2"))
model.add(Conv2D(64, (3, 3),activation='relu',name="conv2d_3"))
model.add(MaxPooling2D(pool_size=(2, 2),name="max_pooling2d_3"))   
model.add(Flatten(name="flatten_1"))
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(128,activation='relu'))          
model.add(Dense(num_classes,activation='softmax'))
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 220, 220, 32)      2432      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 73, 73, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 32)        9248      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 35, 35, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 33, 33, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 16, 16, 64)      

In [13]:
train_datagen=ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
    )
input_shape=(224,224,3)
train_generator =train_datagen.flow_from_directory(train_dir,target_size=(224,224),batch_size=32)
validation_generator = train_datagen.flow_from_directory(test_dir,target_size=(224, 224),batch_size=32)

Found 4721 images belonging to 3 classes.
Found 431 images belonging to 3 classes.


In [16]:
model.compile(optimizer='adam',loss = 'categorical_crossentropy',metrics=['accuracy'])
history1 = model.fit(
    train_generator,#egitim verileri
    steps_per_epoch=None,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=None,
    verbose=1,
    callbacks=[ReduceLROnPlateau(monitor='val_loss', factor=0.3,patience=3, min_lr=0.000001)],
    shuffle=True
    )

Epoch 1/10
148/148 [==============================] - 156s 1s/step - loss: 0.1195 - accuracy: 0.9570 - val_loss: 0.2226 - val_accuracy: 0.9142 - lr: 0.0010
Epoch 2/10
148/148 [==============================] - 154s 1s/step - loss: 0.1016 - accuracy: 0.9634 - val_loss: 0.0965 - val_accuracy: 0.9722 - lr: 0.0010
Epoch 3/10
148/148 [==============================] - 157s 1s/step - loss: 0.0963 - accuracy: 0.9646 - val_loss: 0.1091 - val_accuracy: 0.9582 - lr: 0.0010
Epoch 4/10
148/148 [==============================] - 151s 1s/step - loss: 0.0588 - accuracy: 0.9780 - val_loss: 0.2012 - val_accuracy: 0.9350 - lr: 0.0010
Epoch 5/10
148/148 [==============================] - 150s 1s/step - loss: 0.0887 - accuracy: 0.9670 - val_loss: 0.1087 - val_accuracy: 0.9606 - lr: 0.0010
Epoch 6/10
148/148 [==============================] - 155s 1s/step - loss: 0.0406 - accuracy: 0.9854 - val_loss: 0.0851 - val_accuracy: 0.9652 - lr: 3.0000e-04
Epoch 7/10
148/148 [==============================] - 158s 1

In [17]:
model.save("crop_care.h5")

D:\Anaconda\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
